<a href="https://colab.research.google.com/github/bachaudhry/FastAI-22-23/blob/main/FastAI_2022_Getting_Started_With_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Getting Started with NLP - Using FastAI and Hugging Face**


In [ ]:
import os
import numpy as np
import pandas as pd

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [ ]:
# Setting up Kaggle so that we can download datasets directly
!pip install kaggle

In [ ]:
!pip install transformers[torch]

In [ ]:
# Using Kaggle API key
creds = '{"username":"bachaudhry","key":"f8e0ee26f34cb0d3e4eccd774de62f63"}'

In [ ]:
# Check if file path exists or needs to be created
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
  cred_path.parent.mkdir(exist_ok=True)
  cred_path.write_text(creds)
  cred_path.chmod(0o600)

Now that we have Kaggle setup for this NB, let's download the **US Patent Phrase Matching** dataset.

In [ ]:
path = Path('us-patent-phrase-to-phrase-matching')

if not iskaggle and not path.exists():
  import zipfile, kaggle
  kaggle.api.competition_download_cli(str(path))
  zipfile.ZipFile(f'{path}.zip').extractall(path)

## **Import Data and EDA**

Here's a [description](https://https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/data) for the dataset that we'll be working on.

In short, we will be working on pairs of phrases - which consist of an `anchor` and a `target` phrase.

Additionally, similarity has been scored within a patent's `context`, which is a CPC classification and indicates the subject to which the patent relates.

In [ ]:
# Checking local / GDrive path to verify files in the downloaded dataset.
!ls {path}

sample_submission.csv  test.csv  train.csv


In [ ]:
# Loading training dataset in a DataFrame
df = pd.read_csv(path/'train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36473 entries, 0 to 36472
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       36473 non-null  object 
 1   anchor   36473 non-null  object 
 2   target   36473 non-null  object 
 3   context  36473 non-null  object 
 4   score    36473 non-null  float64
dtypes: float64(1), object(4)
memory usage: 1.4+ MB


In [ ]:
df.head(10)

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
5,067203128142739c,abatement,greenhouse gases,A47,0.25
6,061d17f04be2d1cf,abatement,increased rate,A47,0.25
7,e1f44e48399a2027,abatement,measurement level,A47,0.25
8,0a425937a3e86d10,abatement,minimising sounds,A47,0.50
9,ef2d4c2e6bbb208d,abatement,mixing core materials,A47,0.25


In [ ]:
df.tail(10)

,id,anchor,target,context,score
36463,16a5c8551e534d1c,wood article,wood apple fruit,B44,0.00
36464,8ceaa2b5c2d56250,wood article,wood article,B44,1.00
36465,c4ac9d407fb427ab,wood article,wood logs,B44,0.50
36466,8a57100f6ee40ffc,wood article,wood material,B44,0.75
36467,f55e072f78d1fedb,wood article,wood substrate,B44,0.50
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75
36472,8d135da0b55b8c88,wood article,wooden substrate,B44,0.50


In [ ]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186




We have 36473 rows, 733 unique anchors, 106 contexts and 29340 targets.





In [ ]:
# Creating a column to concatenate the above so that we have a more convenient representation.
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

## Tokenization Using HF Tokenizer

We will be working with HuggingFace's Tokenizer, which uses a `Dataset` object for storage.

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
# This is how a dataset object works when we load in the training DF.
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

It should be noted that Tokenization is dependent on the particular model that we will be using in this notebook. This detail has to be declared in the Tokenizer explicitly.

In [ ]:
# Choosing a small NLP model for exploration
model_nm = 'microsoft/deberta-v3-small'

# Importing HuggingFace Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.


In [ ]:
tokz.tokenize("In the midst of winter, I found there was, within me, an invincible summer.")

['▁In',
 '▁the',
 '▁midst',
 '▁of',
 '▁winter',
 ',',
 '▁I',
 '▁found',
 '▁there',
 '▁was',
 ',',
 '▁within',
 '▁me',
 ',',
 '▁an',
 '▁invincible',
 '▁summer',
 '.']

In [ ]:
tokz.tokenize("amazon.com, is a super-handy website. It's the bee's knees when it comes to e-commerce.")

['▁amazon',
 '.',
 'com',
 ',',
 '▁is',
 '▁a',
 '▁super',
 '-',
 'hand',
 'y',
 '▁website',
 '.',
 '▁It',
 "'",
 's',
 '▁the',
 '▁bee',
 "'",
 's',
 '▁knees',
 '▁when',
 '▁it',
 '▁comes',
 '▁to',
 '▁e',
 '-',
 'commerce',
 '.']

Let's tokenize our inputs column which we created in the previous section.

In [ ]:
# Function to tokenize our inputs.
def tok_func(x):
  return tokz(x["input"])

In [ ]:
# Testing the difference b/w batched=True and the default.
%time tok_ds = ds.map(tok_func)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

CPU times: user 10.3 s, sys: 87.7 ms, total: 10.3 s
Wall time: 10.5 s


In [ ]:
%time tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

CPU times: user 3.82 s, sys: 56.9 ms, total: 3.87 s
Wall time: 3.78 s


The performance gains with the usage of `batched` processing are quite pronounced...

In [ ]:
round((3.8 - 10.5) / ((3.8 + 10.5)/2) * 100, 2)

-93.71

... in this case, we have an improvement which is close to 94%.

Looking at the dataset again, we can see that there is a new column called `input_ids`.

In [ ]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

These IDS come from a list called `vocab` in the tokenizer which contains unique integer for all token strings.

A random example of this can be:

In [ ]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [ ]:
# Checking vocab integers for random words.
tokz.vocab['▁needle'], tokz.vocab['▁the'], tokz.vocab['▁rain']

(9445, 262, 2894)

In [ ]:
# NOTE: Transformers expects the labels column to be named labels by default.
# In our dataset this is refers to the `score` column.
tok_ds = tok_ds.rename_columns({'score': 'labels'})

## Test and Validation Sets

In [ ]:
# Loading test set
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,hybrid bearing,inorganic photoconductor drum,G02
freq,1,2,1,3


The Transformers library uses a `Datasetdict` to hold our training and validation sets. It should be noted that here `validation` is labelled as `test`, so care must be taken.

In [ ]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [ ]:
# We will use eval as our name for the test set.
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
# Now, creating a dataset object based on the changes above.
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

## Training an Initial Model

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
# Selecting a batch size, number of epochs and learning rate
bs = 128
epochs = 20
lr = 8e-5

In [ ]:
# Setting up our training arguments using the class provided by HF
args = TrainingArguments('outputs',
                         learning_rate=lr,
                         warmup_ratio=0.1,
                         lr_scheduler_type='cosine',
                         fp16=True,
                         evaluation_strategy="epoch",
                         per_device_train_batch_size=bs,
                         per_device_eval_batch_size=bs*2,
                         num_train_epochs=epochs,
                         weight_decay=0.01,
                         report_to='none')

In [ ]:
# Function for our evaluation metric i.e. Pearson's Correlation
def corr(x, y):
  return np.corrcoef(x, y)[0][1]

def corr_d(eval_pred):
  return {'pearson': corr(*eval_pred)}

In [ ]:
# Defining the model object and the trainer.
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args,
                  train_dataset=dds['train'],
                  eval_dataset=dds['test'],
                  tokenizer=tokz,
                  compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# Training the model
trainer.train()

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.029663,0.784036
2,No log,0.024111,0.813050
3,0.036600,0.025851,0.824311
4,0.036600,0.024369,0.833777
5,0.015200,0.023721,0.833842
6,0.015200,0.021516,0.838909
7,0.015200,0.020450,0.840711
8,0.009200,0.022619,0.835749
9,0.009200,0.025092,0.836804
10,0.006100,0.021885,0.839774


Checkpoint destination directory outputs/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


TrainOutput(global_step=4280, training_loss=0.009489233262628039, metrics={'train_runtime': 1110.2974, 'train_samples_per_second': 492.733, 'train_steps_per_second': 3.855, 'total_flos': 3582643485809640.0, 'train_loss': 0.009489233262628039, 'epoch': 20.0})

Training for an increased number of epochs doesn't help the model at all.

In fact, the model appears to be overfitting to the training data since training loss decreases are occuring in conjunction with validation loss.

Based on the evaluation metric, Pearson's r doesn't improve beyond the 7th epoch.